In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [3]:
#Understanding how gating works
num_experts = 4
top_k = 2
n_embed=32

In [6]:
# fake multihead attention output
mh_output = torch.randn(2, 4, n_embed) # (B, T, C) = (2, 4, 32) = (batch_size, block_size, n_embed)
mh_output.shape

torch.Size([2, 4, 32])

In [7]:
topkgate_linear = nn.Linear(n_embed, num_experts)
logits = topkgate_linear(mh_output)
logits.shape

torch.Size([2, 4, 4])

In [11]:
logits

tensor([[[ 0.5094,  0.1819,  1.4175, -1.2313],
         [-0.6516, -0.3523,  0.2325, -0.1282],
         [-0.5160,  0.4784,  0.5277,  0.0849],
         [-0.1585,  0.7327, -0.0731,  1.1971]],

        [[-1.0861, -0.4558,  0.8626, -0.2610],
         [ 0.4096, -1.2683,  0.1333, -0.7036],
         [-1.1600,  0.6790,  1.1870, -0.0982],
         [-0.5390,  0.4700, -0.0688,  0.9904]]], grad_fn=<ViewBackward0>)

In [ ]:
# Lấy top_k giá trị lớn nhất và chỉ số của chúng từ logits theo chiều cuối cùng (dim=-1).
top_k_logit, top_k_idx = logits.topk(top_k, dim=-1)


In [10]:
top_k_logit

tensor([[[ 1.4175,  0.5094],
         [ 0.2325, -0.1282],
         [ 0.5277,  0.4784],
         [ 1.1971,  0.7327]],

        [[ 0.8626, -0.2610],
         [ 0.4096,  0.1333],
         [ 1.1870,  0.6790],
         [ 0.9904,  0.4700]]], grad_fn=<TopkBackward0>)

In [12]:
top_k_idx

tensor([[[2, 0],
         [2, 3],
         [2, 1],
         [3, 1]],

        [[2, 3],
         [0, 2],
         [2, 1],
         [3, 1]]])

Lấy đầu ra sparse gating bằng cách chỉ giữ lại k giá trị hàng đầu tại chỉ số tương ứng của chúng dọc theo chiều cuối cùng. Điền phần còn lại bằng ‘-inf’ và truyền qua hàm kích hoạt softmax. Điều này đẩy các giá trị ‘-inf’ về không, làm cho hai giá trị hàng đầu được nhấn mạnh hơn và tổng bằng 1. Tổng bằng 1 này giúp ích cho việc tính trọng số của đầu ra expert.

In [ ]:
# Tạo một tensor zeros có cùng shape với logits (tức (2, 4, num_experts)).
# Các phần tử được điền giá trị âm vô cực (-inf).
# Mục đích là để tạo một mask mà chỉ giữ lại các giá trị top-k.
zeros = torch.full_like(logits, float('-inf'))
zeros

tensor([[[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]],

        [[-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf],
         [-inf, -inf, -inf, -inf]]])

In [ ]:
# Tạo tensor thưa bằng cách đặt các giá trị top_k_logit vào vị trí được chỉ định bởi top_k_idx.
# scatter thay thế các giá trị -inf tại các chỉ số top_k_idx bằng top_k_logit.
# Kết quả sparse_logits có shape (2, 4, num_experts), trong đó:
# Chỉ có top_k giá trị khác -inf ở mỗi chuỗi
# Các vị trí khác vẫn là -inf
sparse_logits = zeros.scatter(-1, top_k_idx, top_k_logit)
sparse_logits

tensor([[[ 0.5094,    -inf,  1.4175,    -inf],
         [   -inf,    -inf,  0.2325, -0.1282],
         [   -inf,  0.4784,  0.5277,    -inf],
         [   -inf,  0.7327,    -inf,  1.1971]],

        [[   -inf,    -inf,  0.8626, -0.2610],
         [ 0.4096,    -inf,  0.1333,    -inf],
         [   -inf,  0.6790,  1.1870,    -inf],
         [   -inf,  0.4700,    -inf,  0.9904]]], grad_fn=<ScatterBackward0>)

In [17]:
gating_output= F.softmax(sparse_logits, dim=-1)
gating_output


tensor([[[0.2874, 0.0000, 0.7126, 0.0000],
         [0.0000, 0.0000, 0.5892, 0.4108],
         [0.0000, 0.4877, 0.5123, 0.0000],
         [0.0000, 0.3860, 0.0000, 0.6140]],

        [[0.0000, 0.0000, 0.7547, 0.2453],
         [0.5686, 0.0000, 0.4314, 0.0000],
         [0.0000, 0.3757, 0.6243, 0.0000],
         [0.0000, 0.3728, 0.0000, 0.6272]]], grad_fn=<SoftmaxBackward0>)

Nếu chỉ dùng top-k sẽ rất có khả năng có những expert được chọn nhiều lần, điều đó dẫn đến không cân bằng tải.

=> Phương pháp: Noisy top-k Gating for load balancing

In [ ]:
# First define the top k router module 
class TopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(TopkRouter, self).__init__()
        self.top_k = top_k
        self.linear =nn.Linear(n_embed, num_experts)
    
    def forward(self, mh_ouput):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.linear(mh_output)
        top_k_logits, indices = logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices
